# Feature Engineering

This notebook will engineer text features from the property reviews, and temporal features from calendar data.  The text features will include the top BOW words from the preprocessed text, and cluster labels after text embedding.  (May need extra computing power).  The temporal features will be aggregated features like average price of neighbourhood by month.

In [1]:
# Read in libraries

In [2]:
# Read in data

In [3]:
# Functions

## Review Feature Engineering

In [ ]:
# BOW

In [ ]:
# Text Embedding

In [ ]:
# Clustering

In [ ]:
# Add new features

## Calendar Feature Engineering

In [ ]:
# Merge neighbourhood data onto calendar data

In [ ]:
# Add day of week feature

In [ ]:
# Add week of year feature

In [ ]:
# Add month of year feature

In [ ]:
# Add year feature

In [ ]:
# Add binary holiday feature

In [ ]:
# Aggregate price by day, week, month, year and neighborhood